# MongoDB Python tutorial. Use https://docs.mongodb.com/getting-started/python

## Set up

In [2]:
import pprint # use this to 'pretty' print data
from pymongo import MongoClient
client = MongoClient()

In [3]:
# To assign the database named primer to the local variable db, you can use attribute access, as in the following:
# db = client.primer
# or
# db = client['primer']

db = client.test

In [4]:
# You can access collection objects directly using dictionary-style or attribute access from a Database object:
coll = db.dataset # or db['dataset']

## Inserting docs

### use the insert_one() or the insert_many() method to add documents to a collection. If collection does not exist, MongoDB will create it.

In [5]:
# Insert a document into a collection named restaurants. The operation will create the collection 
# if the collection does not currently exist.

from datetime import datetime
result = db.restaurants.insert_one(
    {
        "address": {
            "street": "2 Avenue",
            "zipcode": "10075",
            "building": "1480",
            "coord": [-73.9557413, 40.7720266]
        },
        "borough": "Manhattan",
        "cuisine": "Italian",
        "grades": [
            {
                "date": datetime.strptime("2014-10-01", "%Y-%m-%d"),
                "grade": "A",
                "score": 11
            },
            {
                "date": datetime.strptime("2014-01-16", "%Y-%m-%d"),
                "grade": "B",
                "score": 17
            }
        ],
        "name": "Vella",
        "restaurant_id": "41704620"
    }
)

In [6]:
# The operation returns an InsertOneResult object, which includes an attribute inserted_id that contains the 
# _id of the inserted document. Access the inserted_id attribute:
result.inserted_id

ObjectId('58a63657dd1c3f256f7e6f99')

## Find/Query

### Get ALL documents

In [10]:
# this queries for ALL documents in restaurants collection
cursor = db.restaurants.find()
cursor.count()
# do yourself a favor and don't ever print this..
# for document in cursor:
#     pprint.pprint(document)

25360

### top level query

In [11]:
# The following operation finds documents whose borough field equals "Manhattan".

cursor = db.restaurants.find({"borough": "Manhattan"})
cursor.count()
# yeah don't print this one either..
# for document in cursor:
#     pprint.pprint(document)
# The result set includes only the matching documents.

10260

### embedded doc / array search

In [12]:
# use the dot notation for a field in an embedded doc
cursor = db.restaurants.find({"address.zipcode": "10075"})
# cursor = db.restaurants.find({"grades.grade": "B"})

cursor.count()
# for document in cursor:
#     pprint.pprint(document)

100

### Conditional queries

In [17]:
# $gt : greater than
# $lt : less than
cursor = db.restaurants.find({"grades.score": {"$gt": 30}})
print 'grades>30:', cursor.count()

# logical AND
cursor = db.restaurants.find({"cuisine": "Italian", "address.zipcode": "10075"})
print 'Italian food in 10075:', cursor.count()

# logical OR
cursor = db.restaurants.find({"$or": [{"cuisine": "Italian"}, {"address.zipcode": "10075"}]})
print 'either Italian or in 10075:', cursor.count()

grades>30: 1959
Italian food in 10075: 16
either Italian or in 10075: 1154


### Sort queries

In [21]:
import pymongo
cursor = db.restaurants.find(limit=5).sort([
    ("borough", pymongo.ASCENDING),
    ("address.zipcode", pymongo.ASCENDING)
])
for document in cursor:
    pprint.pprint(document)

{u'_id': ObjectId('58a634ba19477c41b841c95e'),
 u'address': {u'building': u'650',
              u'coord': [-73.92537449999999, 40.8207116],
              u'street': u'Grand Concourse',
              u'zipcode': u''},
 u'borough': u'Bronx',
 u'cuisine': u'Sandwiches',
 u'grades': [{u'date': datetime.datetime(2014, 9, 30, 0, 0),
              u'grade': u'A',
              u'score': 7}],
 u'name': u'Subway#50497 (Cardinal Hayes High School)',
 u'restaurant_id': u'50006048'}
{u'_id': ObjectId('58a634b919477c41b8417674'),
 u'address': {u'building': u'72',
              u'coord': [-73.92506, 40.8275556],
              u'street': u'East  161 Street',
              u'zipcode': u'10451'},
 u'borough': u'Bronx',
 u'cuisine': u'American',
 u'grades': [{u'date': datetime.datetime(2014, 4, 15, 0, 0),
              u'grade': u'A',
              u'score': 9},
             {u'date': datetime.datetime(2013, 11, 14, 0, 0),
              u'grade': u'A',
              u'score': 4},
             {u'date': 

## Update entries

### Update a specific field

In [22]:
# to change a field value, use operators such as $set
result = db.restaurants.update_one(
    {"name": "Juni"},
    {
        "$set": {
            "cuisine": "American (New)"
        },
        "$currentDate": {"lastModified": True}
    }
)

In [25]:
print result.matched_count  # number of docs matching the filter condition
print result.modified_count # number of docs modified

1
1


### Update an embedded field

In [26]:
result = db.restaurants.update_one(
    {"restaurant_id": "41156888"},
    {"$set": {"address.street": "East 31st Street"}}
)

In [27]:
print result.matched_count  # number of docs matching the filter condition
print result.modified_count # number of docs modified

1
1


### Update multiple docs

In [28]:
result = db.restaurants.update_many(
    {"address.zipcode": "10016", "cuisine": "Other"},
    {
        "$set": {"cuisine": "Category To Be Determined"},
        "$currentDate": {"lastModified": True}
    }
)

In [29]:
print result.matched_count  # number of docs matching the filter condition
print result.modified_count # number of docs modified

20
20


### Replace a doc

In [30]:
# a document replaces EVERYTHING. It will contain only what is in the replacement_doc
result = db.restaurants.replace_one(
    {"restaurant_id": "41704620"},
    {
        "name": "Vella 2",
        "address": {
            "coord": [-73.9557413, 40.7720266],
            "building": "1480",
            "street": "2 Avenue",
            "zipcode": "10075"
        }
    }
)

In [31]:
print result.matched_count  # number of docs matching the filter condition
print result.modified_count # number of docs modified

1
1


## Remove entries

### Delete entries matching some parameter

In [32]:
result = db.restaurants.delete_many({"borough": "Manhattan"})

In [33]:
result.deleted_count

10259

### Delete all docs

In [34]:
result = db.restaurants.delete_many({})

In [35]:
result.deleted_count

15101

### Delete a collection

In [38]:
# previous operation only removes all docs, not the collection itself
# drop the entire collection, including the indexes
db.restaurants.drop()